In [ ]:
## This file implements neural networks and logistic regression on p0040presabs_quant.
## For fully-connected neural networks, the accuracy is 74.03%, and 79.22% after improvement.
## We further implement the over-sampling method and the combination of over- and under-sampling method, in case of imbalanced data.
## For fully-connected neural networks, the accuracy is 93.55% after regularization for combination data
## and 88.89% for over-sampling data.
## For logistic regression, the accuracies are 77.92% without balanced methods, 98.39% for combination data and 86.87% for over-sampling data.
## For random forest, the accuracy is 76.62% without balanced methods, 98.39% for combination, and 88.89% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 83.69% without balanced methods, 97.24% for combination data, and 83.95% for over-sampling.

In [1]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_quant/p0040presabs_quant.csv')
df.shape

(255, 373)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      0.814000
1      0.365125
2      0.803875
3      1.088125
4      1.081750
5      1.112875
6      0.817875
7      0.398250
8      0.569500
9      0.526125
10     0.351625
11     0.814875
12     0.590875
13     0.373875
14     0.922625
15     0.955750
16     0.383500
17     0.379625
18     0.689625
19     0.186750
20     0.351750
21     0.364250
22     0.306125
23     1.112250
24     0.963000
25     0.159500
26     0.824750
27     0.196000
28     0.590875
29     0.830375
         ...   
225    0.715500
226    0.469000
227    0.670400
228    0.746000
229    0.969000
230    0.577125
231    0.627375
232    0.930500
233    0.478875
234    0.602500
235    0.900125
236    0.646875
237    0.847125
238    0.461500
239    0.258375
240    0.515875
241    0.466875
242    0.275500
243    0.542125
244    0.486625
245    0.765375
246    0.845625
247    0.530875
248    0.894625
249    0.457250
250    0.868000
251    0.550500
252    0.685125
253    0.939500
254    0.926500
Name: pheno, Length: 255

In [5]:
df.head()

,id,TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT,TTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGT,TTTTACAAAAGAGGAAGCAGATAAACTTTATCAACCTATCGGTTCTTCGCAGCCGTCACTGAATATTTGGACAGGCAGTGAAACAGAATATAATTATTTG,TTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTG,TTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTC,TTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTT,TTTGGACAGGCAGTGAAAC,TTTGGACAGGCAGTGAAACAGAATATAATTATTTGTA,TTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATC,...,hxlA_2,group_8822,group_10124,group_1648,group_426,group_8122,group_8213,group_8913,group_9905,pheno
0,107,1,0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0.814000
1,109,0,0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0.365125
2,115,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0.803875
3,120335,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1.088125
4,120337,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1.081750


In [6]:
if 0.5 in df['pheno']:
    print: "0.5 is in the list"

In [7]:
df['pheno'] = [1 if i>0.5 else 0 for i in df['pheno']] # convert pheno into binary

In [8]:
df['pheno']
df['pheno'].value_counts()

1    165
0     90
Name: pheno, dtype: int64

In [9]:
df.shape

(255, 373)

In [10]:
df_clean = df.drop(columns=['id'])

In [11]:
df_clean.shape

(255, 372)

In [12]:
df_clean.head()

,TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT,TTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGT,TTTTACAAAAGAGGAAGCAGATAAACTTTATCAACCTATCGGTTCTTCGCAGCCGTCACTGAATATTTGGACAGGCAGTGAAACAGAATATAATTATTTG,TTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTG,TTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTC,TTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTT,TTTGGACAGGCAGTGAAAC,TTTGGACAGGCAGTGAAACAGAATATAATTATTTGTA,TTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATC,TTTCGCTGTTGCAACATCTTTAACAGGTGTTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACT,...,hxlA_2,group_8822,group_10124,group_1648,group_426,group_8122,group_8213,group_8913,group_9905,pheno
0,1,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
3,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
4,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1


In [13]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 371) (255,)


In [14]:
# split into train, test data (original)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y)

In [15]:
############# Fully-Connected Neural Network ################

In [16]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

Using TensorFlow backend.


In [17]:
#### neural network
model1 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [18]:
model1.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [19]:
model1.fit(X_train, y_train,
          batch_size=32, epochs=1000,
          validation_data=(X_test, y_test))

Train on 178 samples, validate on 77 samples
Epoch 1/1000
178/178 [==============================] - 0s 625us/step - loss: 0.6608 - accuracy: 0.6461 - val_loss: 0.6247 - val_accuracy: 0.6753
Epoch 2/1000
178/178 [==============================] - 0s 105us/step - loss: 0.5679 - accuracy: 0.6910 - val_loss: 0.5954 - val_accuracy: 0.6883
Epoch 3/1000
178/178 [==============================] - 0s 157us/step - loss: 0.5203 - accuracy: 0.7640 - val_loss: 0.5812 - val_accuracy: 0.7013
Epoch 4/1000
178/178 [==============================] - 0s 152us/step - loss: 0.4921 - accuracy: 0.7528 - val_loss: 0.5745 - val_accuracy: 0.6883
Epoch 5/1000
178/178 [==============================] - 0s 132us/step - loss: 0.4744 - accuracy: 0.7472 - val_loss: 0.5695 - val_accuracy: 0.6883
Epoch 6/1000
178/178 [==============================] - 0s 171us/step - loss: 0.4637 - accuracy: 0.7640 - val_loss: 0.5675 - val_accuracy: 0.6883
Epoch 7/1000
178/178 [==============================] - 0s 169us/step - loss: 0

Epoch 112/1000
178/178 [==============================] - 0s 101us/step - loss: 0.3026 - accuracy: 0.8933 - val_loss: 0.5524 - val_accuracy: 0.7273
Epoch 113/1000
178/178 [==============================] - 0s 110us/step - loss: 0.2997 - accuracy: 0.8820 - val_loss: 0.5532 - val_accuracy: 0.7403
Epoch 114/1000
178/178 [==============================] - 0s 147us/step - loss: 0.3010 - accuracy: 0.8989 - val_loss: 0.5507 - val_accuracy: 0.7403
Epoch 115/1000
178/178 [==============================] - 0s 109us/step - loss: 0.2984 - accuracy: 0.8933 - val_loss: 0.5522 - val_accuracy: 0.7403
Epoch 116/1000
178/178 [==============================] - 0s 109us/step - loss: 0.2973 - accuracy: 0.8989 - val_loss: 0.5515 - val_accuracy: 0.7403
Epoch 117/1000
178/178 [==============================] - 0s 101us/step - loss: 0.2956 - accuracy: 0.8933 - val_loss: 0.5531 - val_accuracy: 0.7403
Epoch 118/1000
178/178 [==============================] - 0s 137us/step - loss: 0.2967 - accuracy: 0.8989 - val_

178/178 [==============================] - 0s 103us/step - loss: 0.2290 - accuracy: 0.9157 - val_loss: 0.5757 - val_accuracy: 0.7792
Epoch 223/1000
178/178 [==============================] - 0s 98us/step - loss: 0.2291 - accuracy: 0.9213 - val_loss: 0.5780 - val_accuracy: 0.7792
Epoch 224/1000
178/178 [==============================] - 0s 112us/step - loss: 0.2303 - accuracy: 0.9101 - val_loss: 0.5769 - val_accuracy: 0.7662
Epoch 225/1000
178/178 [==============================] - 0s 106us/step - loss: 0.2291 - accuracy: 0.9213 - val_loss: 0.5780 - val_accuracy: 0.7662
Epoch 226/1000
178/178 [==============================] - 0s 96us/step - loss: 0.2285 - accuracy: 0.9157 - val_loss: 0.5786 - val_accuracy: 0.7662
Epoch 227/1000
178/178 [==============================] - 0s 105us/step - loss: 0.2272 - accuracy: 0.9157 - val_loss: 0.5817 - val_accuracy: 0.7792
Epoch 228/1000
178/178 [==============================] - 0s 96us/step - loss: 0.2273 - accuracy: 0.9213 - val_loss: 0.5855 - val

178/178 [==============================] - 0s 102us/step - loss: 0.1807 - accuracy: 0.9382 - val_loss: 0.6135 - val_accuracy: 0.7662
Epoch 333/1000
178/178 [==============================] - 0s 100us/step - loss: 0.1804 - accuracy: 0.9326 - val_loss: 0.6139 - val_accuracy: 0.7532
Epoch 334/1000
178/178 [==============================] - 0s 126us/step - loss: 0.1807 - accuracy: 0.9382 - val_loss: 0.6175 - val_accuracy: 0.7532
Epoch 335/1000
178/178 [==============================] - 0s 102us/step - loss: 0.1796 - accuracy: 0.9382 - val_loss: 0.6190 - val_accuracy: 0.7532
Epoch 336/1000
178/178 [==============================] - 0s 100us/step - loss: 0.1822 - accuracy: 0.9382 - val_loss: 0.6142 - val_accuracy: 0.7662
Epoch 337/1000
178/178 [==============================] - 0s 131us/step - loss: 0.1783 - accuracy: 0.9382 - val_loss: 0.6141 - val_accuracy: 0.7792
Epoch 338/1000
178/178 [==============================] - 0s 172us/step - loss: 0.1788 - accuracy: 0.9438 - val_loss: 0.6157 - 

178/178 [==============================] - 0s 105us/step - loss: 0.1457 - accuracy: 0.9607 - val_loss: 0.6611 - val_accuracy: 0.7662
Epoch 443/1000
178/178 [==============================] - 0s 101us/step - loss: 0.1469 - accuracy: 0.9663 - val_loss: 0.6608 - val_accuracy: 0.7792
Epoch 444/1000
178/178 [==============================] - 0s 107us/step - loss: 0.1439 - accuracy: 0.9607 - val_loss: 0.6662 - val_accuracy: 0.7662
Epoch 445/1000
178/178 [==============================] - 0s 115us/step - loss: 0.1463 - accuracy: 0.9551 - val_loss: 0.6654 - val_accuracy: 0.7662
Epoch 446/1000
178/178 [==============================] - 0s 104us/step - loss: 0.1431 - accuracy: 0.9607 - val_loss: 0.6690 - val_accuracy: 0.7662
Epoch 447/1000
178/178 [==============================] - 0s 110us/step - loss: 0.1446 - accuracy: 0.9551 - val_loss: 0.6677 - val_accuracy: 0.7662
Epoch 448/1000
178/178 [==============================] - 0s 112us/step - loss: 0.1432 - accuracy: 0.9551 - val_loss: 0.6633 - 

178/178 [==============================] - 0s 96us/step - loss: 0.1202 - accuracy: 0.9719 - val_loss: 0.7145 - val_accuracy: 0.7662
Epoch 553/1000
178/178 [==============================] - 0s 102us/step - loss: 0.1208 - accuracy: 0.9663 - val_loss: 0.7077 - val_accuracy: 0.7662
Epoch 554/1000
178/178 [==============================] - 0s 87us/step - loss: 0.1212 - accuracy: 0.9719 - val_loss: 0.7143 - val_accuracy: 0.7662
Epoch 555/1000
178/178 [==============================] - 0s 136us/step - loss: 0.1215 - accuracy: 0.9719 - val_loss: 0.7099 - val_accuracy: 0.7662
Epoch 556/1000
178/178 [==============================] - 0s 104us/step - loss: 0.1201 - accuracy: 0.9719 - val_loss: 0.7140 - val_accuracy: 0.7662
Epoch 557/1000
178/178 [==============================] - 0s 98us/step - loss: 0.1222 - accuracy: 0.9719 - val_loss: 0.7130 - val_accuracy: 0.7662
Epoch 558/1000
178/178 [==============================] - 0s 89us/step - loss: 0.1188 - accuracy: 0.9719 - val_loss: 0.7165 - val_

178/178 [==============================] - 0s 104us/step - loss: 0.1043 - accuracy: 0.9719 - val_loss: 0.7556 - val_accuracy: 0.7662
Epoch 663/1000
178/178 [==============================] - 0s 109us/step - loss: 0.1033 - accuracy: 0.9719 - val_loss: 0.7543 - val_accuracy: 0.7662
Epoch 664/1000
178/178 [==============================] - 0s 112us/step - loss: 0.1041 - accuracy: 0.9719 - val_loss: 0.7558 - val_accuracy: 0.7662
Epoch 665/1000
178/178 [==============================] - 0s 108us/step - loss: 0.1031 - accuracy: 0.9719 - val_loss: 0.7596 - val_accuracy: 0.7662
Epoch 666/1000
178/178 [==============================] - 0s 106us/step - loss: 0.1046 - accuracy: 0.9719 - val_loss: 0.7581 - val_accuracy: 0.7662
Epoch 667/1000
178/178 [==============================] - 0s 103us/step - loss: 0.1021 - accuracy: 0.9719 - val_loss: 0.7597 - val_accuracy: 0.7662
Epoch 668/1000
178/178 [==============================] - 0s 101us/step - loss: 0.1044 - accuracy: 0.9719 - val_loss: 0.7637 - 

178/178 [==============================] - 0s 89us/step - loss: 0.0923 - accuracy: 0.9719 - val_loss: 0.8052 - val_accuracy: 0.7532
Epoch 773/1000
178/178 [==============================] - 0s 107us/step - loss: 0.0903 - accuracy: 0.9719 - val_loss: 0.8026 - val_accuracy: 0.7532
Epoch 774/1000
178/178 [==============================] - 0s 99us/step - loss: 0.0912 - accuracy: 0.9719 - val_loss: 0.8041 - val_accuracy: 0.7532
Epoch 775/1000
178/178 [==============================] - 0s 108us/step - loss: 0.0890 - accuracy: 0.9719 - val_loss: 0.8069 - val_accuracy: 0.7532
Epoch 776/1000
178/178 [==============================] - 0s 89us/step - loss: 0.0891 - accuracy: 0.9719 - val_loss: 0.8034 - val_accuracy: 0.7532
Epoch 777/1000
178/178 [==============================] - 0s 95us/step - loss: 0.0902 - accuracy: 0.9719 - val_loss: 0.8039 - val_accuracy: 0.7662
Epoch 778/1000
178/178 [==============================] - 0s 95us/step - loss: 0.0894 - accuracy: 0.9775 - val_loss: 0.8096 - val_a

178/178 [==============================] - 0s 114us/step - loss: 0.0804 - accuracy: 0.9775 - val_loss: 0.8397 - val_accuracy: 0.7532
Epoch 883/1000
178/178 [==============================] - 0s 121us/step - loss: 0.0798 - accuracy: 0.9775 - val_loss: 0.8435 - val_accuracy: 0.7532
Epoch 884/1000
178/178 [==============================] - 0s 113us/step - loss: 0.0811 - accuracy: 0.9775 - val_loss: 0.8422 - val_accuracy: 0.7532
Epoch 885/1000
178/178 [==============================] - 0s 100us/step - loss: 0.0794 - accuracy: 0.9719 - val_loss: 0.8414 - val_accuracy: 0.7532
Epoch 886/1000
178/178 [==============================] - 0s 122us/step - loss: 0.0794 - accuracy: 0.9775 - val_loss: 0.8457 - val_accuracy: 0.7532
Epoch 887/1000
178/178 [==============================] - 0s 103us/step - loss: 0.0809 - accuracy: 0.9775 - val_loss: 0.8464 - val_accuracy: 0.7532
Epoch 888/1000
178/178 [==============================] - 0s 118us/step - loss: 0.0794 - accuracy: 0.9775 - val_loss: 0.8480 - 

178/178 [==============================] - 0s 151us/step - loss: 0.0741 - accuracy: 0.9719 - val_loss: 0.8738 - val_accuracy: 0.7273
Epoch 993/1000
178/178 [==============================] - 0s 167us/step - loss: 0.0757 - accuracy: 0.9831 - val_loss: 0.8792 - val_accuracy: 0.7403
Epoch 994/1000
178/178 [==============================] - 0s 180us/step - loss: 0.0778 - accuracy: 0.9719 - val_loss: 0.8837 - val_accuracy: 0.7532
Epoch 995/1000
178/178 [==============================] - 0s 126us/step - loss: 0.0736 - accuracy: 0.9775 - val_loss: 0.8916 - val_accuracy: 0.7532
Epoch 996/1000
178/178 [==============================] - 0s 159us/step - loss: 0.0749 - accuracy: 0.9719 - val_loss: 0.8864 - val_accuracy: 0.7532
Epoch 997/1000
178/178 [==============================] - 0s 169us/step - loss: 0.0741 - accuracy: 0.9719 - val_loss: 0.8915 - val_accuracy: 0.7532
Epoch 998/1000
178/178 [==============================] - 0s 173us/step - loss: 0.0736 - accuracy: 0.9719 - val_loss: 0.8871 - 

In [20]:
acc_test1 = model1.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test1*100))

77/77 [==============================] - 0s 146us/step
test accuracy: 74.03%


In [27]:
#### improve neural network
model2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [28]:
model2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [29]:
model2.fit(X_train, y_train,
          batch_size=64, epochs=1000,
          validation_data=(X_test, y_test))

Train on 178 samples, validate on 77 samples
Epoch 1/1000
178/178 [==============================] - 0s 652us/step - loss: 0.7350 - accuracy: 0.5000 - val_loss: 0.6835 - val_accuracy: 0.6364
Epoch 2/1000
178/178 [==============================] - 0s 79us/step - loss: 0.6654 - accuracy: 0.6011 - val_loss: 0.6488 - val_accuracy: 0.6623
Epoch 3/1000
178/178 [==============================] - 0s 87us/step - loss: 0.6202 - accuracy: 0.7416 - val_loss: 0.6229 - val_accuracy: 0.7273
Epoch 4/1000
178/178 [==============================] - 0s 97us/step - loss: 0.5867 - accuracy: 0.8202 - val_loss: 0.6064 - val_accuracy: 0.7403
Epoch 5/1000
178/178 [==============================] - 0s 122us/step - loss: 0.5626 - accuracy: 0.8315 - val_loss: 0.5931 - val_accuracy: 0.7792
Epoch 6/1000
178/178 [==============================] - 0s 103us/step - loss: 0.5441 - accuracy: 0.8315 - val_loss: 0.5842 - val_accuracy: 0.7792
Epoch 7/1000
178/178 [==============================] - 0s 118us/step - loss: 0.53

Epoch 112/1000
178/178 [==============================] - 0s 87us/step - loss: 0.3739 - accuracy: 0.8652 - val_loss: 0.5324 - val_accuracy: 0.7662
Epoch 113/1000
178/178 [==============================] - 0s 94us/step - loss: 0.3738 - accuracy: 0.8596 - val_loss: 0.5322 - val_accuracy: 0.7662
Epoch 114/1000
178/178 [==============================] - 0s 126us/step - loss: 0.3731 - accuracy: 0.8596 - val_loss: 0.5328 - val_accuracy: 0.7662
Epoch 115/1000
178/178 [==============================] - 0s 193us/step - loss: 0.3728 - accuracy: 0.8596 - val_loss: 0.5329 - val_accuracy: 0.7662
Epoch 116/1000
178/178 [==============================] - 0s 77us/step - loss: 0.3721 - accuracy: 0.8596 - val_loss: 0.5324 - val_accuracy: 0.7662
Epoch 117/1000
178/178 [==============================] - 0s 97us/step - loss: 0.3717 - accuracy: 0.8652 - val_loss: 0.5329 - val_accuracy: 0.7662
Epoch 118/1000
178/178 [==============================] - 0s 94us/step - loss: 0.3723 - accuracy: 0.8652 - val_loss:

178/178 [==============================] - 0s 72us/step - loss: 0.3301 - accuracy: 0.8876 - val_loss: 0.5295 - val_accuracy: 0.7792
Epoch 223/1000
178/178 [==============================] - 0s 89us/step - loss: 0.3316 - accuracy: 0.8820 - val_loss: 0.5290 - val_accuracy: 0.7662
Epoch 224/1000
178/178 [==============================] - 0s 89us/step - loss: 0.3295 - accuracy: 0.8876 - val_loss: 0.5294 - val_accuracy: 0.7792
Epoch 225/1000
178/178 [==============================] - 0s 87us/step - loss: 0.3298 - accuracy: 0.8820 - val_loss: 0.5300 - val_accuracy: 0.7792
Epoch 226/1000
178/178 [==============================] - 0s 83us/step - loss: 0.3286 - accuracy: 0.8876 - val_loss: 0.5301 - val_accuracy: 0.7792
Epoch 227/1000
178/178 [==============================] - 0s 85us/step - loss: 0.3286 - accuracy: 0.8876 - val_loss: 0.5305 - val_accuracy: 0.7792
Epoch 228/1000
178/178 [==============================] - 0s 87us/step - loss: 0.3287 - accuracy: 0.8876 - val_loss: 0.5301 - val_acc

Epoch 278/1000
178/178 [==============================] - 0s 87us/step - loss: 0.3084 - accuracy: 0.8933 - val_loss: 0.5315 - val_accuracy: 0.7662
Epoch 279/1000
178/178 [==============================] - 0s 75us/step - loss: 0.3097 - accuracy: 0.8876 - val_loss: 0.5337 - val_accuracy: 0.7922
Epoch 280/1000
178/178 [==============================] - 0s 116us/step - loss: 0.3079 - accuracy: 0.8876 - val_loss: 0.5316 - val_accuracy: 0.7662
Epoch 281/1000
178/178 [==============================] - 0s 73us/step - loss: 0.3074 - accuracy: 0.8933 - val_loss: 0.5316 - val_accuracy: 0.7662
Epoch 282/1000
178/178 [==============================] - 0s 76us/step - loss: 0.3072 - accuracy: 0.8876 - val_loss: 0.5322 - val_accuracy: 0.7792
Epoch 283/1000
178/178 [==============================] - 0s 86us/step - loss: 0.3070 - accuracy: 0.8933 - val_loss: 0.5324 - val_accuracy: 0.7792
Epoch 284/1000
178/178 [==============================] - 0s 71us/step - loss: 0.3090 - accuracy: 0.8933 - val_loss: 

Epoch 334/1000
178/178 [==============================] - 0s 78us/step - loss: 0.2853 - accuracy: 0.8989 - val_loss: 0.5333 - val_accuracy: 0.7662
Epoch 335/1000
178/178 [==============================] - 0s 80us/step - loss: 0.2842 - accuracy: 0.8989 - val_loss: 0.5328 - val_accuracy: 0.7662
Epoch 336/1000
178/178 [==============================] - 0s 64us/step - loss: 0.2844 - accuracy: 0.8989 - val_loss: 0.5329 - val_accuracy: 0.7662
Epoch 337/1000
178/178 [==============================] - 0s 88us/step - loss: 0.2829 - accuracy: 0.8989 - val_loss: 0.5333 - val_accuracy: 0.7662
Epoch 338/1000
178/178 [==============================] - 0s 85us/step - loss: 0.2833 - accuracy: 0.8989 - val_loss: 0.5326 - val_accuracy: 0.7662
Epoch 339/1000
178/178 [==============================] - 0s 82us/step - loss: 0.2833 - accuracy: 0.8989 - val_loss: 0.5335 - val_accuracy: 0.7792
Epoch 340/1000
178/178 [==============================] - 0s 77us/step - loss: 0.2823 - accuracy: 0.8989 - val_loss: 0

...

178/178 [==============================] - 0s 74us/step - loss: 0.1423 - accuracy: 0.9494 - val_loss: 0.6129 - val_accuracy: 0.7922
Epoch 722/1000
178/178 [==============================] - 0s 73us/step - loss: 0.1398 - accuracy: 0.9494 - val_loss: 0.6159 - val_accuracy: 0.7922
Epoch 723/1000
178/178 [==============================] - 0s 71us/step - loss: 0.1390 - accuracy: 0.9494 - val_loss: 0.6121 - val_accuracy: 0.7922
Epoch 724/1000
178/178 [==============================] - 0s 98us/step - loss: 0.1396 - accuracy: 0.9551 - val_loss: 0.6164 - val_accuracy: 0.7922
Epoch 725/1000
178/178 [==============================] - ETA: 0s - loss: 0.1340 - accuracy: 0.96 - 0s 178us/step - loss: 0.1399 - accuracy: 0.9551 - val_loss: 0.6178 - val_accuracy: 0.7922
Epoch 726/1000
178/178 [==============================] - 0s 89us/step - loss: 0.1408 - accuracy: 0.9494 - val_loss: 0.6108 - val_accuracy: 0.7922
Epoch 727/1000
178/178 [==============================] - 0s 81us/step - loss: 0.1385 - ac

Epoch 832/1000
178/178 [==============================] - 0s 87us/step - loss: 0.1144 - accuracy: 0.9663 - val_loss: 0.6587 - val_accuracy: 0.7922
Epoch 833/1000
178/178 [==============================] - 0s 89us/step - loss: 0.1133 - accuracy: 0.9607 - val_loss: 0.6557 - val_accuracy: 0.7922
Epoch 834/1000
178/178 [==============================] - 0s 86us/step - loss: 0.1143 - accuracy: 0.9663 - val_loss: 0.6571 - val_accuracy: 0.7922
Epoch 835/1000
178/178 [==============================] - 0s 90us/step - loss: 0.1128 - accuracy: 0.9663 - val_loss: 0.6613 - val_accuracy: 0.7922
Epoch 836/1000
178/178 [==============================] - 0s 82us/step - loss: 0.1123 - accuracy: 0.9663 - val_loss: 0.6576 - val_accuracy: 0.7922
Epoch 837/1000
178/178 [==============================] - 0s 104us/step - loss: 0.1121 - accuracy: 0.9663 - val_loss: 0.6664 - val_accuracy: 0.7922
Epoch 838/1000
178/178 [==============================] - 0s 83us/step - loss: 0.1119 - accuracy: 0.9663 - val_loss: 

Epoch 888/1000
178/178 [==============================] - 0s 72us/step - loss: 0.1047 - accuracy: 0.9663 - val_loss: 0.6908 - val_accuracy: 0.7922
Epoch 889/1000
178/178 [==============================] - 0s 90us/step - loss: 0.1005 - accuracy: 0.9663 - val_loss: 0.6856 - val_accuracy: 0.7922
Epoch 890/1000
178/178 [==============================] - 0s 94us/step - loss: 0.1016 - accuracy: 0.9719 - val_loss: 0.6860 - val_accuracy: 0.7922
Epoch 891/1000
178/178 [==============================] - 0s 87us/step - loss: 0.1004 - accuracy: 0.9719 - val_loss: 0.6881 - val_accuracy: 0.7922
Epoch 892/1000
178/178 [==============================] - 0s 96us/step - loss: 0.1009 - accuracy: 0.9719 - val_loss: 0.6838 - val_accuracy: 0.7922
Epoch 893/1000
178/178 [==============================] - 0s 97us/step - loss: 0.1058 - accuracy: 0.9719 - val_loss: 0.6901 - val_accuracy: 0.7922
Epoch 894/1000
178/178 [==============================] - 0s 77us/step - loss: 0.1047 - accuracy: 0.9663 - val_loss: 0

Epoch 944/1000
178/178 [==============================] - 0s 83us/step - loss: 0.0926 - accuracy: 0.9719 - val_loss: 0.7067 - val_accuracy: 0.7922
Epoch 945/1000
178/178 [==============================] - 0s 77us/step - loss: 0.0923 - accuracy: 0.9719 - val_loss: 0.7045 - val_accuracy: 0.7922
Epoch 946/1000
178/178 [==============================] - 0s 85us/step - loss: 0.0914 - accuracy: 0.9719 - val_loss: 0.7171 - val_accuracy: 0.7922
Epoch 947/1000
178/178 [==============================] - 0s 89us/step - loss: 0.0919 - accuracy: 0.9663 - val_loss: 0.7113 - val_accuracy: 0.7922
Epoch 948/1000
178/178 [==============================] - 0s 72us/step - loss: 0.0910 - accuracy: 0.9719 - val_loss: 0.7150 - val_accuracy: 0.7922
Epoch 949/1000
178/178 [==============================] - 0s 74us/step - loss: 0.0915 - accuracy: 0.9663 - val_loss: 0.7101 - val_accuracy: 0.7922
Epoch 950/1000
178/178 [==============================] - 0s 66us/step - loss: 0.0920 - accuracy: 0.9719 - val_loss: 0

Epoch 1000/1000
178/178 [==============================] - 0s 70us/step - loss: 0.0840 - accuracy: 0.9663 - val_loss: 0.7365 - val_accuracy: 0.7922


In [30]:
acc_test2 = model2.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test2*100))

77/77 [==============================] - 0s 134us/step
test accuracy: 79.22%


In [31]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

[(0, 108), (1, 98)]


In [32]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 165), (1, 165)]


In [33]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [34]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [35]:
############# Fully-Connected Neural Network ################

In [36]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [37]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [38]:
model1_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [39]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 144 samples, validate on 62 samples
Epoch 1/1000
144/144 [==============================] - 0s 731us/step - loss: 0.7997 - accuracy: 0.3125 - val_loss: 0.7632 - val_accuracy: 0.4677
Epoch 2/1000
144/144 [==============================] - 0s 126us/step - loss: 0.6624 - accuracy: 0.6181 - val_loss: 0.6694 - val_accuracy: 0.5806
Epoch 3/1000
144/144 [==============================] - 0s 145us/step - loss: 0.5790 - accuracy: 0.7014 - val_loss: 0.5997 - val_accuracy: 0.6290
Epoch 4/1000
144/144 [==============================] - 0s 142us/step - loss: 0.5187 - accuracy: 0.7500 - val_loss: 0.5447 - val_accuracy: 0.7097
Epoch 5/1000
144/144 [==============================] - 0s 158us/step - loss: 0.4734 - accuracy: 0.7639 - val_loss: 0.4999 - val_accuracy: 0.7903
Epoch 6/1000
144/144 [==============================] - 0s 151us/step - loss: 0.4368 - accuracy: 0.7778 - val_loss: 0.4642 - val_accuracy: 0.7903
Epoch 7/1000
144/144 [==============================] - 0s 143us/step - loss: 0

144/144 [==============================] - 0s 99us/step - loss: 0.1054 - accuracy: 0.9722 - val_loss: 0.1295 - val_accuracy: 0.9516
Epoch 112/1000
144/144 [==============================] - 0s 106us/step - loss: 0.1051 - accuracy: 0.9722 - val_loss: 0.1281 - val_accuracy: 0.9516
Epoch 113/1000
144/144 [==============================] - 0s 89us/step - loss: 0.1040 - accuracy: 0.9722 - val_loss: 0.1277 - val_accuracy: 0.9516
Epoch 114/1000
144/144 [==============================] - 0s 126us/step - loss: 0.1034 - accuracy: 0.9722 - val_loss: 0.1280 - val_accuracy: 0.9516
Epoch 115/1000
144/144 [==============================] - 0s 129us/step - loss: 0.1026 - accuracy: 0.9722 - val_loss: 0.1275 - val_accuracy: 0.9516
Epoch 116/1000
144/144 [==============================] - 0s 129us/step - loss: 0.1023 - accuracy: 0.9722 - val_loss: 0.1280 - val_accuracy: 0.9516
Epoch 117/1000
144/144 [==============================] - 0s 144us/step - loss: 0.1013 - accuracy: 0.9722 - val_loss: 0.1269 - va

144/144 [==============================] - 0s 120us/step - loss: 0.0546 - accuracy: 0.9931 - val_loss: 0.0963 - val_accuracy: 0.9677
Epoch 222/1000
144/144 [==============================] - 0s 120us/step - loss: 0.0542 - accuracy: 0.9931 - val_loss: 0.0953 - val_accuracy: 0.9677
Epoch 223/1000
144/144 [==============================] - 0s 121us/step - loss: 0.0539 - accuracy: 0.9931 - val_loss: 0.0951 - val_accuracy: 0.9677
Epoch 224/1000
144/144 [==============================] - 0s 136us/step - loss: 0.0536 - accuracy: 0.9931 - val_loss: 0.0956 - val_accuracy: 0.9677
Epoch 225/1000
144/144 [==============================] - ETA: 0s - loss: 0.0401 - accuracy: 1.00 - 0s 118us/step - loss: 0.0533 - accuracy: 0.9931 - val_loss: 0.0959 - val_accuracy: 0.9677
Epoch 226/1000
144/144 [==============================] - 0s 115us/step - loss: 0.0532 - accuracy: 0.9931 - val_loss: 0.0957 - val_accuracy: 0.9677
Epoch 227/1000
144/144 [==============================] - 0s 116us/step - loss: 0.052

144/144 [==============================] - 0s 114us/step - loss: 0.0328 - accuracy: 1.0000 - val_loss: 0.0787 - val_accuracy: 0.9839
Epoch 332/1000
144/144 [==============================] - 0s 162us/step - loss: 0.0326 - accuracy: 1.0000 - val_loss: 0.0786 - val_accuracy: 0.9839
Epoch 333/1000
144/144 [==============================] - 0s 116us/step - loss: 0.0325 - accuracy: 1.0000 - val_loss: 0.0788 - val_accuracy: 0.9839
Epoch 334/1000
144/144 [==============================] - 0s 110us/step - loss: 0.0326 - accuracy: 1.0000 - val_loss: 0.0792 - val_accuracy: 0.9839
Epoch 335/1000
144/144 [==============================] - 0s 109us/step - loss: 0.0323 - accuracy: 1.0000 - val_loss: 0.0803 - val_accuracy: 0.9839
Epoch 336/1000
144/144 [==============================] - 0s 101us/step - loss: 0.0323 - accuracy: 1.0000 - val_loss: 0.0797 - val_accuracy: 0.9839
Epoch 337/1000
144/144 [==============================] - 0s 121us/step - loss: 0.0319 - accuracy: 1.0000 - val_loss: 0.0791 - 

144/144 [==============================] - 0s 103us/step - loss: 0.0219 - accuracy: 1.0000 - val_loss: 0.0704 - val_accuracy: 0.9839
Epoch 442/1000
144/144 [==============================] - 0s 104us/step - loss: 0.0219 - accuracy: 1.0000 - val_loss: 0.0701 - val_accuracy: 0.9839
Epoch 443/1000
144/144 [==============================] - 0s 110us/step - loss: 0.0218 - accuracy: 1.0000 - val_loss: 0.0698 - val_accuracy: 0.9839
Epoch 444/1000
144/144 [==============================] - 0s 202us/step - loss: 0.0218 - accuracy: 1.0000 - val_loss: 0.0699 - val_accuracy: 0.9839
Epoch 445/1000
144/144 [==============================] - 0s 119us/step - loss: 0.0217 - accuracy: 1.0000 - val_loss: 0.0707 - val_accuracy: 0.9839
Epoch 446/1000
144/144 [==============================] - 0s 109us/step - loss: 0.0218 - accuracy: 1.0000 - val_loss: 0.0704 - val_accuracy: 0.9839
Epoch 447/1000
144/144 [==============================] - 0s 141us/step - loss: 0.0216 - accuracy: 1.0000 - val_loss: 0.0698 - 

144/144 [==============================] - 0s 155us/step - loss: 0.0160 - accuracy: 1.0000 - val_loss: 0.0634 - val_accuracy: 0.9839
Epoch 552/1000
144/144 [==============================] - 0s 157us/step - loss: 0.0160 - accuracy: 1.0000 - val_loss: 0.0642 - val_accuracy: 0.9839
Epoch 553/1000
144/144 [==============================] - 0s 145us/step - loss: 0.0159 - accuracy: 1.0000 - val_loss: 0.0642 - val_accuracy: 0.9839
Epoch 554/1000
144/144 [==============================] - 0s 114us/step - loss: 0.0159 - accuracy: 1.0000 - val_loss: 0.0635 - val_accuracy: 0.9839
Epoch 555/1000
144/144 [==============================] - 0s 113us/step - loss: 0.0158 - accuracy: 1.0000 - val_loss: 0.0638 - val_accuracy: 0.9839
Epoch 556/1000
144/144 [==============================] - 0s 145us/step - loss: 0.0159 - accuracy: 1.0000 - val_loss: 0.0638 - val_accuracy: 0.9839
Epoch 557/1000
144/144 [==============================] - 0s 136us/step - loss: 0.0158 - accuracy: 1.0000 - val_loss: 0.0644 - 

144/144 [==============================] - 0s 101us/step - loss: 0.0123 - accuracy: 1.0000 - val_loss: 0.0609 - val_accuracy: 0.9839
Epoch 662/1000
144/144 [==============================] - 0s 91us/step - loss: 0.0122 - accuracy: 1.0000 - val_loss: 0.0607 - val_accuracy: 0.9839
Epoch 663/1000
144/144 [==============================] - ETA: 0s - loss: 0.0136 - accuracy: 1.00 - 0s 114us/step - loss: 0.0122 - accuracy: 1.0000 - val_loss: 0.0607 - val_accuracy: 0.9839
Epoch 664/1000
144/144 [==============================] - 0s 107us/step - loss: 0.0122 - accuracy: 1.0000 - val_loss: 0.0606 - val_accuracy: 0.9839
Epoch 665/1000
144/144 [==============================] - 0s 100us/step - loss: 0.0122 - accuracy: 1.0000 - val_loss: 0.0604 - val_accuracy: 0.9839
Epoch 666/1000
144/144 [==============================] - 0s 106us/step - loss: 0.0122 - accuracy: 1.0000 - val_loss: 0.0602 - val_accuracy: 0.9839
Epoch 667/1000
144/144 [==============================] - 0s 132us/step - loss: 0.0121

144/144 [==============================] - 0s 106us/step - loss: 0.0099 - accuracy: 1.0000 - val_loss: 0.0582 - val_accuracy: 0.9839
Epoch 772/1000
144/144 [==============================] - 0s 118us/step - loss: 0.0098 - accuracy: 1.0000 - val_loss: 0.0582 - val_accuracy: 0.9839
Epoch 773/1000
144/144 [==============================] - 0s 111us/step - loss: 0.0097 - accuracy: 1.0000 - val_loss: 0.0582 - val_accuracy: 0.9839
Epoch 774/1000
144/144 [==============================] - 0s 122us/step - loss: 0.0098 - accuracy: 1.0000 - val_loss: 0.0582 - val_accuracy: 0.9839
Epoch 775/1000
144/144 [==============================] - 0s 125us/step - loss: 0.0097 - accuracy: 1.0000 - val_loss: 0.0580 - val_accuracy: 0.9839
Epoch 776/1000
144/144 [==============================] - 0s 124us/step - loss: 0.0097 - accuracy: 1.0000 - val_loss: 0.0580 - val_accuracy: 0.9839
Epoch 777/1000
144/144 [==============================] - 0s 181us/step - loss: 0.0097 - accuracy: 1.0000 - val_loss: 0.0579 - 

144/144 [==============================] - 0s 101us/step - loss: 0.0080 - accuracy: 1.0000 - val_loss: 0.0567 - val_accuracy: 0.9839
Epoch 882/1000
144/144 [==============================] - 0s 113us/step - loss: 0.0080 - accuracy: 1.0000 - val_loss: 0.0568 - val_accuracy: 0.9839
Epoch 883/1000
144/144 [==============================] - 0s 139us/step - loss: 0.0080 - accuracy: 1.0000 - val_loss: 0.0566 - val_accuracy: 0.9839
Epoch 884/1000
144/144 [==============================] - 0s 138us/step - loss: 0.0080 - accuracy: 1.0000 - val_loss: 0.0566 - val_accuracy: 0.9839
Epoch 885/1000
144/144 [==============================] - 0s 112us/step - loss: 0.0080 - accuracy: 1.0000 - val_loss: 0.0567 - val_accuracy: 0.9839
Epoch 886/1000
144/144 [==============================] - 0s 124us/step - loss: 0.0080 - accuracy: 1.0000 - val_loss: 0.0568 - val_accuracy: 0.9839
Epoch 887/1000
144/144 [==============================] - 0s 133us/step - loss: 0.0080 - accuracy: 1.0000 - val_loss: 0.0565 - 

144/144 [==============================] - 0s 239us/step - loss: 0.0068 - accuracy: 1.0000 - val_loss: 0.0553 - val_accuracy: 0.9839
Epoch 992/1000
144/144 [==============================] - 0s 115us/step - loss: 0.0067 - accuracy: 1.0000 - val_loss: 0.0554 - val_accuracy: 0.9839
Epoch 993/1000
144/144 [==============================] - 0s 144us/step - loss: 0.0068 - accuracy: 1.0000 - val_loss: 0.0553 - val_accuracy: 0.9839
Epoch 994/1000
144/144 [==============================] - 0s 126us/step - loss: 0.0067 - accuracy: 1.0000 - val_loss: 0.0556 - val_accuracy: 0.9839
Epoch 995/1000
144/144 [==============================] - 0s 109us/step - loss: 0.0067 - accuracy: 1.0000 - val_loss: 0.0556 - val_accuracy: 0.9839
Epoch 996/1000
144/144 [==============================] - 0s 122us/step - loss: 0.0067 - accuracy: 1.0000 - val_loss: 0.0553 - val_accuracy: 0.9839
Epoch 997/1000
144/144 [==============================] - 0s 118us/step - loss: 0.0067 - accuracy: 1.0000 - val_loss: 0.0553 - 

In [40]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

62/62 [==============================] - 0s 131us/step
combination test accuracy: 98.39%


In [41]:
#### add regularizor and dropout
model1_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [42]:
model1_comb2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [43]:
model1_comb2.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 144 samples, validate on 62 samples
Epoch 1/1000
144/144 [==============================] - 0s 1ms/step - loss: 1.8619 - accuracy: 0.5764 - val_loss: 0.7891 - val_accuracy: 0.8871
Epoch 2/1000
144/144 [==============================] - 0s 123us/step - loss: 2.4786 - accuracy: 0.6875 - val_loss: 0.7198 - val_accuracy: 0.8871
Epoch 3/1000
144/144 [==============================] - 0s 210us/step - loss: 1.1409 - accuracy: 0.8542 - val_loss: 0.6661 - val_accuracy: 0.9032
Epoch 4/1000
144/144 [==============================] - 0s 150us/step - loss: 1.7152 - accuracy: 0.8333 - val_loss: 0.6294 - val_accuracy: 0.9032
Epoch 5/1000
144/144 [==============================] - 0s 188us/step - loss: 1.5855 - accuracy: 0.8264 - val_loss: 0.5991 - val_accuracy: 0.9032
Epoch 6/1000
144/144 [==============================] - 0s 220us/step - loss: 2.2892 - accuracy: 0.8056 - val_loss: 0.5756 - val_accuracy: 0.9032
Epoch 7/1000
144/144 [==============================] - 0s 159us/step - loss: 2.4

Epoch 112/1000
144/144 [==============================] - 0s 151us/step - loss: 1.1761 - accuracy: 0.8958 - val_loss: 0.3317 - val_accuracy: 0.9516
Epoch 113/1000
144/144 [==============================] - 0s 149us/step - loss: 1.0775 - accuracy: 0.8958 - val_loss: 0.3307 - val_accuracy: 0.9516
Epoch 114/1000
144/144 [==============================] - 0s 182us/step - loss: 1.6053 - accuracy: 0.8611 - val_loss: 0.3305 - val_accuracy: 0.9355
Epoch 115/1000
144/144 [==============================] - 0s 152us/step - loss: 1.5812 - accuracy: 0.8750 - val_loss: 0.3294 - val_accuracy: 0.9355
Epoch 116/1000
144/144 [==============================] - 0s 200us/step - loss: 2.1362 - accuracy: 0.8264 - val_loss: 0.3342 - val_accuracy: 0.9516
Epoch 117/1000
144/144 [==============================] - 0s 178us/step - loss: 1.7170 - accuracy: 0.8472 - val_loss: 0.3337 - val_accuracy: 0.9677
Epoch 118/1000
144/144 [==============================] - 0s 155us/step - loss: 2.2329 - accuracy: 0.8264 - val_

144/144 [==============================] - 0s 122us/step - loss: 1.3642 - accuracy: 0.8819 - val_loss: 0.3077 - val_accuracy: 0.9839
Epoch 168/1000
144/144 [==============================] - 0s 121us/step - loss: 1.2416 - accuracy: 0.9028 - val_loss: 0.3051 - val_accuracy: 0.9839
Epoch 169/1000
144/144 [==============================] - 0s 131us/step - loss: 1.0358 - accuracy: 0.9028 - val_loss: 0.3062 - val_accuracy: 0.9839
Epoch 170/1000
144/144 [==============================] - 0s 128us/step - loss: 1.6824 - accuracy: 0.8611 - val_loss: 0.3082 - val_accuracy: 0.9839
Epoch 171/1000
144/144 [==============================] - 0s 127us/step - loss: 2.0985 - accuracy: 0.8472 - val_loss: 0.3079 - val_accuracy: 0.9839
Epoch 172/1000
144/144 [==============================] - 0s 122us/step - loss: 1.6661 - accuracy: 0.8819 - val_loss: 0.3052 - val_accuracy: 0.9839
Epoch 173/1000
144/144 [==============================] - 0s 131us/step - loss: 1.5482 - accuracy: 0.8819 - val_loss: 0.3030 - 

144/144 [==============================] - 0s 114us/step - loss: 1.5222 - accuracy: 0.9028 - val_loss: 0.2678 - val_accuracy: 0.9839
Epoch 278/1000
144/144 [==============================] - 0s 124us/step - loss: 1.4142 - accuracy: 0.9097 - val_loss: 0.2668 - val_accuracy: 0.9839
Epoch 279/1000
144/144 [==============================] - 0s 150us/step - loss: 1.2024 - accuracy: 0.9236 - val_loss: 0.2661 - val_accuracy: 0.9839
Epoch 280/1000
144/144 [==============================] - 0s 163us/step - loss: 1.3082 - accuracy: 0.9167 - val_loss: 0.2677 - val_accuracy: 0.9839
Epoch 281/1000
144/144 [==============================] - 0s 160us/step - loss: 1.5277 - accuracy: 0.9028 - val_loss: 0.2666 - val_accuracy: 0.9839
Epoch 282/1000
144/144 [==============================] - 0s 134us/step - loss: 1.7412 - accuracy: 0.8889 - val_loss: 0.2655 - val_accuracy: 0.9839
Epoch 283/1000
144/144 [==============================] - 0s 152us/step - loss: 1.4173 - accuracy: 0.9097 - val_loss: 0.2664 - 

144/144 [==============================] - 0s 101us/step - loss: 1.2892 - accuracy: 0.9236 - val_loss: 0.2473 - val_accuracy: 0.9839
Epoch 388/1000
144/144 [==============================] - 0s 229us/step - loss: 1.2883 - accuracy: 0.9236 - val_loss: 0.2457 - val_accuracy: 0.9839
Epoch 389/1000
144/144 [==============================] - 0s 130us/step - loss: 1.9188 - accuracy: 0.8819 - val_loss: 0.2462 - val_accuracy: 0.9839
Epoch 390/1000
144/144 [==============================] - 0s 121us/step - loss: 1.7080 - accuracy: 0.8958 - val_loss: 0.2434 - val_accuracy: 0.9839
Epoch 391/1000
144/144 [==============================] - 0s 125us/step - loss: 1.7078 - accuracy: 0.8958 - val_loss: 0.2463 - val_accuracy: 0.9839
Epoch 392/1000
144/144 [==============================] - 0s 125us/step - loss: 1.8224 - accuracy: 0.8889 - val_loss: 0.2502 - val_accuracy: 0.9677
Epoch 393/1000
144/144 [==============================] - 0s 162us/step - loss: 1.9242 - accuracy: 0.8819 - val_loss: 0.2437 - 

144/144 [==============================] - 0s 119us/step - loss: 1.1666 - accuracy: 0.9306 - val_loss: 0.2371 - val_accuracy: 0.9516
Epoch 498/1000
144/144 [==============================] - 0s 121us/step - loss: 1.2721 - accuracy: 0.9236 - val_loss: 0.2393 - val_accuracy: 0.9516
Epoch 499/1000
144/144 [==============================] - 0s 119us/step - loss: 1.7030 - accuracy: 0.8958 - val_loss: 0.2387 - val_accuracy: 0.9516
Epoch 500/1000
144/144 [==============================] - 0s 117us/step - loss: 1.4874 - accuracy: 0.9097 - val_loss: 0.2348 - val_accuracy: 0.9677
Epoch 501/1000
144/144 [==============================] - 0s 129us/step - loss: 1.4847 - accuracy: 0.9097 - val_loss: 0.2328 - val_accuracy: 0.9677
Epoch 502/1000
144/144 [==============================] - 0s 108us/step - loss: 1.4859 - accuracy: 0.9097 - val_loss: 0.2330 - val_accuracy: 0.9677
Epoch 503/1000
144/144 [==============================] - 0s 118us/step - loss: 2.0205 - accuracy: 0.8750 - val_loss: 0.2356 - 

144/144 [==============================] - 0s 123us/step - loss: 1.4791 - accuracy: 0.9097 - val_loss: 0.2324 - val_accuracy: 0.9516
Epoch 608/1000
144/144 [==============================] - 0s 127us/step - loss: 1.0486 - accuracy: 0.9375 - val_loss: 0.2298 - val_accuracy: 0.9516
Epoch 609/1000
144/144 [==============================] - 0s 127us/step - loss: 1.1536 - accuracy: 0.9306 - val_loss: 0.2318 - val_accuracy: 0.9516
Epoch 610/1000
144/144 [==============================] - 0s 139us/step - loss: 1.9014 - accuracy: 0.8819 - val_loss: 0.2322 - val_accuracy: 0.9516
Epoch 611/1000
144/144 [==============================] - 0s 144us/step - loss: 1.1528 - accuracy: 0.9306 - val_loss: 0.2306 - val_accuracy: 0.9516
Epoch 612/1000
144/144 [==============================] - 0s 173us/step - loss: 2.0119 - accuracy: 0.8750 - val_loss: 0.2304 - val_accuracy: 0.9516
Epoch 613/1000
144/144 [==============================] - 0s 126us/step - loss: 1.7966 - accuracy: 0.8889 - val_loss: 0.2331 - 

144/144 [==============================] - 0s 122us/step - loss: 1.3615 - accuracy: 0.9167 - val_loss: 0.2223 - val_accuracy: 0.9516
Epoch 718/1000
144/144 [==============================] - 0s 148us/step - loss: 1.6845 - accuracy: 0.8958 - val_loss: 0.2216 - val_accuracy: 0.9516
Epoch 719/1000
144/144 [==============================] - 0s 141us/step - loss: 2.5392 - accuracy: 0.8403 - val_loss: 0.2226 - val_accuracy: 0.9516
Epoch 720/1000
144/144 [==============================] - 0s 134us/step - loss: 1.4716 - accuracy: 0.9097 - val_loss: 0.2327 - val_accuracy: 0.9516
Epoch 721/1000
144/144 [==============================] - 0s 130us/step - loss: 1.6840 - accuracy: 0.8958 - val_loss: 0.2260 - val_accuracy: 0.9516
Epoch 722/1000
144/144 [==============================] - 0s 138us/step - loss: 1.7929 - accuracy: 0.8889 - val_loss: 0.2272 - val_accuracy: 0.9516
Epoch 723/1000
144/144 [==============================] - 0s 132us/step - loss: 1.3624 - accuracy: 0.9167 - val_loss: 0.2220 - 

144/144 [==============================] - 0s 149us/step - loss: 1.3588 - accuracy: 0.9167 - val_loss: 0.2235 - val_accuracy: 0.9516
Epoch 828/1000
144/144 [==============================] - 0s 193us/step - loss: 1.2516 - accuracy: 0.9236 - val_loss: 0.2188 - val_accuracy: 0.9516
Epoch 829/1000
144/144 [==============================] - 0s 170us/step - loss: 1.7854 - accuracy: 0.8889 - val_loss: 0.2234 - val_accuracy: 0.9516
Epoch 830/1000
144/144 [==============================] - 0s 138us/step - loss: 1.0358 - accuracy: 0.9375 - val_loss: 0.2234 - val_accuracy: 0.9516
Epoch 831/1000
144/144 [==============================] - 0s 160us/step - loss: 1.7847 - accuracy: 0.8889 - val_loss: 0.2170 - val_accuracy: 0.9516
Epoch 832/1000
144/144 [==============================] - 0s 150us/step - loss: 1.2493 - accuracy: 0.9236 - val_loss: 0.2224 - val_accuracy: 0.9516
Epoch 833/1000
144/144 [==============================] - 0s 148us/step - loss: 1.1414 - accuracy: 0.9306 - val_loss: 0.2229 - 

144/144 [==============================] - 0s 121us/step - loss: 1.6754 - accuracy: 0.8958 - val_loss: 0.2226 - val_accuracy: 0.9355
Epoch 938/1000
144/144 [==============================] - 0s 117us/step - loss: 1.4579 - accuracy: 0.9097 - val_loss: 0.2212 - val_accuracy: 0.9516
Epoch 939/1000
144/144 [==============================] - 0s 124us/step - loss: 1.6745 - accuracy: 0.8958 - val_loss: 0.2221 - val_accuracy: 0.9355
Epoch 940/1000
144/144 [==============================] - 0s 125us/step - loss: 0.7089 - accuracy: 0.9583 - val_loss: 0.2185 - val_accuracy: 0.9516
Epoch 941/1000
144/144 [==============================] - 0s 173us/step - loss: 1.6728 - accuracy: 0.8958 - val_loss: 0.2207 - val_accuracy: 0.9355
Epoch 942/1000
144/144 [==============================] - 0s 134us/step - loss: 1.8880 - accuracy: 0.8819 - val_loss: 0.2204 - val_accuracy: 0.9355
Epoch 943/1000
144/144 [==============================] - 0s 118us/step - loss: 1.2450 - accuracy: 0.9236 - val_loss: 0.2208 - 

In [44]:
acc_test_comb2 = model1_comb2.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))

62/62 [==============================] - 0s 132us/step
combination test accuracy: 93.55%


In [45]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [46]:
model1_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [47]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 231 samples, validate on 99 samples
Epoch 1/1000
231/231 [==============================] - 0s 461us/step - loss: 0.7794 - accuracy: 0.5325 - val_loss: 0.6859 - val_accuracy: 0.5657
Epoch 2/1000
231/231 [==============================] - 0s 123us/step - loss: 0.6827 - accuracy: 0.5714 - val_loss: 0.6394 - val_accuracy: 0.6768
Epoch 3/1000
231/231 [==============================] - 0s 138us/step - loss: 0.6348 - accuracy: 0.6623 - val_loss: 0.6176 - val_accuracy: 0.7172
Epoch 4/1000
231/231 [==============================] - 0s 136us/step - loss: 0.6094 - accuracy: 0.7100 - val_loss: 0.5948 - val_accuracy: 0.7273
Epoch 5/1000
231/231 [==============================] - 0s 130us/step - loss: 0.5904 - accuracy: 0.7229 - val_loss: 0.5828 - val_accuracy: 0.7576
Epoch 6/1000
231/231 [==============================] - 0s 141us/step - loss: 0.5723 - accuracy: 0.7359 - val_loss: 0.5654 - val_accuracy: 0.7374
Epoch 7/1000
231/231 [==============================] - 0s 132us/step - loss: 0

Epoch 112/1000
231/231 [==============================] - 0s 112us/step - loss: 0.3553 - accuracy: 0.8658 - val_loss: 0.4365 - val_accuracy: 0.8283
Epoch 113/1000
231/231 [==============================] - 0s 161us/step - loss: 0.3512 - accuracy: 0.8528 - val_loss: 0.4337 - val_accuracy: 0.8384
Epoch 114/1000
231/231 [==============================] - 0s 280us/step - loss: 0.3537 - accuracy: 0.8571 - val_loss: 0.4330 - val_accuracy: 0.8182
Epoch 115/1000
231/231 [==============================] - 0s 499us/step - loss: 0.3480 - accuracy: 0.8571 - val_loss: 0.4349 - val_accuracy: 0.8283
Epoch 116/1000
231/231 [==============================] - 0s 192us/step - loss: 0.3482 - accuracy: 0.8615 - val_loss: 0.4400 - val_accuracy: 0.8384
Epoch 117/1000
231/231 [==============================] - 0s 162us/step - loss: 0.3485 - accuracy: 0.8615 - val_loss: 0.4378 - val_accuracy: 0.8586
Epoch 118/1000
231/231 [==============================] - 0s 121us/step - loss: 0.3463 - accuracy: 0.8658 - val_

231/231 [==============================] - 0s 179us/step - loss: 0.2617 - accuracy: 0.9091 - val_loss: 0.4049 - val_accuracy: 0.8586
Epoch 223/1000
231/231 [==============================] - ETA: 0s - loss: 0.2823 - accuracy: 0.87 - 0s 186us/step - loss: 0.2635 - accuracy: 0.9048 - val_loss: 0.4253 - val_accuracy: 0.8788
Epoch 224/1000
231/231 [==============================] - 0s 162us/step - loss: 0.2601 - accuracy: 0.9134 - val_loss: 0.4054 - val_accuracy: 0.8687
Epoch 225/1000
231/231 [==============================] - 0s 109us/step - loss: 0.2602 - accuracy: 0.9091 - val_loss: 0.4079 - val_accuracy: 0.8586
Epoch 226/1000
231/231 [==============================] - 0s 94us/step - loss: 0.2602 - accuracy: 0.9048 - val_loss: 0.4072 - val_accuracy: 0.8586
Epoch 227/1000
231/231 [==============================] - 0s 103us/step - loss: 0.2593 - accuracy: 0.8961 - val_loss: 0.4314 - val_accuracy: 0.8687
Epoch 228/1000
231/231 [==============================] - 0s 95us/step - loss: 0.2610 

Epoch 277/1000
231/231 [==============================] - 0s 187us/step - loss: 0.2348 - accuracy: 0.9221 - val_loss: 0.4404 - val_accuracy: 0.8182
Epoch 278/1000
231/231 [==============================] - 0s 153us/step - loss: 0.2321 - accuracy: 0.9351 - val_loss: 0.4031 - val_accuracy: 0.8687
Epoch 279/1000
231/231 [==============================] - 0s 129us/step - loss: 0.2274 - accuracy: 0.9351 - val_loss: 0.3974 - val_accuracy: 0.8687
Epoch 280/1000
231/231 [==============================] - 0s 107us/step - loss: 0.2317 - accuracy: 0.9394 - val_loss: 0.4053 - val_accuracy: 0.8586
Epoch 281/1000
231/231 [==============================] - 0s 120us/step - loss: 0.2278 - accuracy: 0.9307 - val_loss: 0.4502 - val_accuracy: 0.8384
Epoch 282/1000
231/231 [==============================] - 0s 124us/step - loss: 0.2315 - accuracy: 0.9048 - val_loss: 0.4786 - val_accuracy: 0.8081
Epoch 283/1000
231/231 [==============================] - 0s 119us/step - loss: 0.2313 - accuracy: 0.9177 - val_

Epoch 388/1000
231/231 [==============================] - 0s 96us/step - loss: 0.1810 - accuracy: 0.9524 - val_loss: 0.4208 - val_accuracy: 0.8687
Epoch 389/1000
231/231 [==============================] - 0s 96us/step - loss: 0.1833 - accuracy: 0.9481 - val_loss: 0.4185 - val_accuracy: 0.8788
Epoch 390/1000
231/231 [==============================] - 0s 93us/step - loss: 0.1817 - accuracy: 0.9524 - val_loss: 0.3960 - val_accuracy: 0.8687
Epoch 391/1000
231/231 [==============================] - 0s 94us/step - loss: 0.1887 - accuracy: 0.9481 - val_loss: 0.4171 - val_accuracy: 0.8788
Epoch 392/1000
231/231 [==============================] - 0s 100us/step - loss: 0.1800 - accuracy: 0.9481 - val_loss: 0.4096 - val_accuracy: 0.8889
Epoch 393/1000
231/231 [==============================] - 0s 99us/step - loss: 0.1803 - accuracy: 0.9524 - val_loss: 0.4161 - val_accuracy: 0.8889
Epoch 394/1000
231/231 [==============================] - 0s 104us/step - loss: 0.1779 - accuracy: 0.9524 - val_loss:

...

231/231 [==============================] - 0s 121us/step - loss: 0.1279 - accuracy: 0.9697 - val_loss: 0.4303 - val_accuracy: 0.8889
Epoch 609/1000
231/231 [==============================] - 0s 99us/step - loss: 0.1272 - accuracy: 0.9697 - val_loss: 0.4144 - val_accuracy: 0.8788
Epoch 610/1000
231/231 [==============================] - 0s 114us/step - loss: 0.1299 - accuracy: 0.9697 - val_loss: 0.4436 - val_accuracy: 0.8788
Epoch 611/1000
231/231 [==============================] - 0s 102us/step - loss: 0.1262 - accuracy: 0.9697 - val_loss: 0.4423 - val_accuracy: 0.8788
Epoch 612/1000
231/231 [==============================] - 0s 100us/step - loss: 0.1298 - accuracy: 0.9610 - val_loss: 0.4800 - val_accuracy: 0.8384
Epoch 613/1000
231/231 [==============================] - 0s 97us/step - loss: 0.1294 - accuracy: 0.9610 - val_loss: 0.4348 - val_accuracy: 0.8889
Epoch 614/1000
231/231 [==============================] - 0s 107us/step - loss: 0.1258 - accuracy: 0.9697 - val_loss: 0.4139 - va

Epoch 719/1000
231/231 [==============================] - 0s 196us/step - loss: 0.1119 - accuracy: 0.9697 - val_loss: 0.4268 - val_accuracy: 0.8788
Epoch 720/1000
231/231 [==============================] - 0s 101us/step - loss: 0.1132 - accuracy: 0.9697 - val_loss: 0.4244 - val_accuracy: 0.8586
Epoch 721/1000
231/231 [==============================] - 0s 104us/step - loss: 0.1214 - accuracy: 0.9697 - val_loss: 0.4649 - val_accuracy: 0.8687
Epoch 722/1000
231/231 [==============================] - 0s 95us/step - loss: 0.1128 - accuracy: 0.9697 - val_loss: 0.4574 - val_accuracy: 0.8788
Epoch 723/1000
231/231 [==============================] - 0s 114us/step - loss: 0.1112 - accuracy: 0.9697 - val_loss: 0.4723 - val_accuracy: 0.8586
Epoch 724/1000
231/231 [==============================] - 0s 108us/step - loss: 0.1153 - accuracy: 0.9654 - val_loss: 0.4440 - val_accuracy: 0.8889
Epoch 725/1000
231/231 [==============================] - 0s 112us/step - loss: 0.1139 - accuracy: 0.9697 - val_l

231/231 [==============================] - 0s 95us/step - loss: 0.1050 - accuracy: 0.9610 - val_loss: 0.4777 - val_accuracy: 0.8687
Epoch 830/1000
231/231 [==============================] - 0s 110us/step - loss: 0.1026 - accuracy: 0.9697 - val_loss: 0.4625 - val_accuracy: 0.8889
Epoch 831/1000
231/231 [==============================] - 0s 102us/step - loss: 0.1017 - accuracy: 0.9697 - val_loss: 0.5107 - val_accuracy: 0.8081
Epoch 832/1000
231/231 [==============================] - 0s 92us/step - loss: 0.0978 - accuracy: 0.9697 - val_loss: 0.4397 - val_accuracy: 0.8687
Epoch 833/1000
231/231 [==============================] - 0s 134us/step - loss: 0.1062 - accuracy: 0.9697 - val_loss: 0.4546 - val_accuracy: 0.8889
Epoch 834/1000
231/231 [==============================] - 0s 96us/step - loss: 0.1041 - accuracy: 0.9654 - val_loss: 0.4762 - val_accuracy: 0.8687
Epoch 835/1000
231/231 [==============================] - 0s 98us/step - loss: 0.1027 - accuracy: 0.9610 - val_loss: 0.4712 - val_

Epoch 885/1000
231/231 [==============================] - 0s 90us/step - loss: 0.0943 - accuracy: 0.9654 - val_loss: 0.4895 - val_accuracy: 0.8485
Epoch 886/1000
231/231 [==============================] - 0s 89us/step - loss: 0.0959 - accuracy: 0.9697 - val_loss: 0.4734 - val_accuracy: 0.8788
Epoch 887/1000
231/231 [==============================] - 0s 99us/step - loss: 0.0984 - accuracy: 0.9697 - val_loss: 0.4617 - val_accuracy: 0.8889
Epoch 888/1000
231/231 [==============================] - 0s 93us/step - loss: 0.0954 - accuracy: 0.9697 - val_loss: 0.4534 - val_accuracy: 0.8889
Epoch 889/1000
231/231 [==============================] - 0s 106us/step - loss: 0.0974 - accuracy: 0.9697 - val_loss: 0.4336 - val_accuracy: 0.8485
Epoch 890/1000
231/231 [==============================] - 0s 94us/step - loss: 0.0980 - accuracy: 0.9697 - val_loss: 0.4791 - val_accuracy: 0.8485
Epoch 891/1000
231/231 [==============================] - 0s 110us/step - loss: 0.0976 - accuracy: 0.9697 - val_loss:

231/231 [==============================] - 0s 89us/step - loss: 0.0971 - accuracy: 0.9654 - val_loss: 0.4925 - val_accuracy: 0.8485
Epoch 996/1000
231/231 [==============================] - 0s 105us/step - loss: 0.0905 - accuracy: 0.9654 - val_loss: 0.5177 - val_accuracy: 0.8384
Epoch 997/1000
231/231 [==============================] - 0s 98us/step - loss: 0.0934 - accuracy: 0.9567 - val_loss: 0.4898 - val_accuracy: 0.8687
Epoch 998/1000
231/231 [==============================] - 0s 107us/step - loss: 0.0865 - accuracy: 0.9654 - val_loss: 0.4834 - val_accuracy: 0.8687
Epoch 999/1000
231/231 [==============================] - 0s 98us/step - loss: 0.0942 - accuracy: 0.9567 - val_loss: 0.4755 - val_accuracy: 0.8889
Epoch 1000/1000
231/231 [==============================] - 0s 107us/step - loss: 0.0875 - accuracy: 0.9654 - val_loss: 0.4721 - val_accuracy: 0.8889


In [48]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

99/99 [==============================] - 0s 108us/step
over-sampling test accuracy: 88.89%


In [49]:
#### add regularizer and dropout on over-sampling data
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [50]:
model1_over2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [51]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 231 samples, validate on 99 samples
Epoch 1/1000
231/231 [==============================] - 0s 735us/step - loss: 2.2486 - accuracy: 0.5455 - val_loss: 0.8948 - val_accuracy: 0.4949
Epoch 2/1000
231/231 [==============================] - 0s 109us/step - loss: 2.3727 - accuracy: 0.6190 - val_loss: 0.8273 - val_accuracy: 0.5354
Epoch 3/1000
231/231 [==============================] - 0s 157us/step - loss: 2.1952 - accuracy: 0.6797 - val_loss: 0.7776 - val_accuracy: 0.5354
Epoch 4/1000
231/231 [==============================] - 0s 118us/step - loss: 2.6012 - accuracy: 0.6970 - val_loss: 0.7497 - val_accuracy: 0.5758
Epoch 5/1000
231/231 [==============================] - 0s 136us/step - loss: 2.1420 - accuracy: 0.6926 - val_loss: 0.7309 - val_accuracy: 0.5960
Epoch 6/1000
231/231 [==============================] - 0s 137us/step - loss: 2.0404 - accuracy: 0.7143 - val_loss: 0.7080 - val_accuracy: 0.7071
Epoch 7/1000
231/231 [==============================] - 0s 131us/step - loss: 2

Epoch 112/1000
231/231 [==============================] - 0s 157us/step - loss: 1.6937 - accuracy: 0.7922 - val_loss: 0.5392 - val_accuracy: 0.8182
Epoch 113/1000
231/231 [==============================] - 0s 96us/step - loss: 1.2298 - accuracy: 0.8225 - val_loss: 0.5327 - val_accuracy: 0.8081
Epoch 114/1000
231/231 [==============================] - 0s 105us/step - loss: 1.5753 - accuracy: 0.7965 - val_loss: 0.5334 - val_accuracy: 0.7980
Epoch 115/1000
231/231 [==============================] - 0s 115us/step - loss: 2.0256 - accuracy: 0.7749 - val_loss: 0.5352 - val_accuracy: 0.8182
Epoch 116/1000
231/231 [==============================] - 0s 99us/step - loss: 2.0896 - accuracy: 0.7706 - val_loss: 0.5609 - val_accuracy: 0.7677
Epoch 117/1000
231/231 [==============================] - 0s 115us/step - loss: 1.5099 - accuracy: 0.7965 - val_loss: 0.5445 - val_accuracy: 0.8283
Epoch 118/1000
231/231 [==============================] - 0s 99us/step - loss: 1.8206 - accuracy: 0.7965 - val_los

231/231 [==============================] - 0s 112us/step - loss: 2.5346 - accuracy: 0.7792 - val_loss: 0.5086 - val_accuracy: 0.8384
Epoch 223/1000
231/231 [==============================] - 0s 126us/step - loss: 2.0875 - accuracy: 0.8009 - val_loss: 0.5005 - val_accuracy: 0.8283
Epoch 224/1000
231/231 [==============================] - 0s 113us/step - loss: 1.7647 - accuracy: 0.8139 - val_loss: 0.5023 - val_accuracy: 0.8283
Epoch 225/1000
231/231 [==============================] - 0s 107us/step - loss: 1.4293 - accuracy: 0.8442 - val_loss: 0.5008 - val_accuracy: 0.8283
Epoch 226/1000
231/231 [==============================] - 0s 130us/step - loss: 2.0860 - accuracy: 0.7965 - val_loss: 0.5030 - val_accuracy: 0.8384
Epoch 227/1000
231/231 [==============================] - 0s 138us/step - loss: 1.8950 - accuracy: 0.8095 - val_loss: 0.5338 - val_accuracy: 0.7980
Epoch 228/1000
231/231 [==============================] - 0s 116us/step - loss: 1.7950 - accuracy: 0.8355 - val_loss: 0.5076 - 

231/231 [==============================] - 0s 125us/step - loss: 2.1719 - accuracy: 0.8095 - val_loss: 0.5058 - val_accuracy: 0.8384
Epoch 333/1000
231/231 [==============================] - 0s 133us/step - loss: 1.8992 - accuracy: 0.8312 - val_loss: 0.5012 - val_accuracy: 0.8485
Epoch 334/1000
231/231 [==============================] - 0s 131us/step - loss: 1.3134 - accuracy: 0.8658 - val_loss: 0.5476 - val_accuracy: 0.8081
Epoch 335/1000
231/231 [==============================] - 0s 127us/step - loss: 1.6474 - accuracy: 0.8312 - val_loss: 0.4941 - val_accuracy: 0.8485
Epoch 336/1000
231/231 [==============================] - 0s 110us/step - loss: 1.5672 - accuracy: 0.8658 - val_loss: 0.5000 - val_accuracy: 0.8485
Epoch 337/1000
231/231 [==============================] - 0s 123us/step - loss: 1.1851 - accuracy: 0.8658 - val_loss: 0.4970 - val_accuracy: 0.8485
Epoch 338/1000
231/231 [==============================] - 0s 184us/step - loss: 1.7143 - accuracy: 0.8398 - val_loss: 0.4985 - 

231/231 [==============================] - 0s 104us/step - loss: 1.7391 - accuracy: 0.8615 - val_loss: 0.5289 - val_accuracy: 0.8384
Epoch 443/1000
231/231 [==============================] - 0s 103us/step - loss: 1.5312 - accuracy: 0.8701 - val_loss: 0.5047 - val_accuracy: 0.8485
Epoch 444/1000
231/231 [==============================] - 0s 100us/step - loss: 1.5446 - accuracy: 0.8658 - val_loss: 0.5061 - val_accuracy: 0.8485
Epoch 445/1000
231/231 [==============================] - 0s 99us/step - loss: 1.0822 - accuracy: 0.8961 - val_loss: 0.5061 - val_accuracy: 0.8485
Epoch 446/1000
231/231 [==============================] - 0s 95us/step - loss: 1.6142 - accuracy: 0.8658 - val_loss: 0.5239 - val_accuracy: 0.8485
Epoch 447/1000
231/231 [==============================] - 0s 103us/step - loss: 1.4178 - accuracy: 0.8745 - val_loss: 0.5162 - val_accuracy: 0.8485
Epoch 448/1000
231/231 [==============================] - 0s 109us/step - loss: 1.4066 - accuracy: 0.8831 - val_loss: 0.5139 - va

231/231 [==============================] - 0s 106us/step - loss: 1.5348 - accuracy: 0.8658 - val_loss: 0.5218 - val_accuracy: 0.8586
Epoch 553/1000
231/231 [==============================] - 0s 105us/step - loss: 2.1727 - accuracy: 0.8442 - val_loss: 0.5106 - val_accuracy: 0.8586
Epoch 554/1000
231/231 [==============================] - 0s 108us/step - loss: 1.8556 - accuracy: 0.8485 - val_loss: 0.5456 - val_accuracy: 0.8586
Epoch 555/1000
231/231 [==============================] - 0s 102us/step - loss: 1.6631 - accuracy: 0.8658 - val_loss: 0.5187 - val_accuracy: 0.8586
Epoch 556/1000
231/231 [==============================] - 0s 104us/step - loss: 2.1216 - accuracy: 0.8355 - val_loss: 0.5247 - val_accuracy: 0.8586
Epoch 557/1000
231/231 [==============================] - 0s 106us/step - loss: 1.7834 - accuracy: 0.8528 - val_loss: 0.5272 - val_accuracy: 0.8586
Epoch 558/1000
231/231 [==============================] - 0s 105us/step - loss: 1.5236 - accuracy: 0.8701 - val_loss: 0.5198 - 

231/231 [==============================] - 0s 151us/step - loss: 1.7130 - accuracy: 0.8528 - val_loss: 0.5361 - val_accuracy: 0.8586
Epoch 663/1000
231/231 [==============================] - 0s 101us/step - loss: 1.9741 - accuracy: 0.8398 - val_loss: 0.5479 - val_accuracy: 0.8586
Epoch 664/1000
231/231 [==============================] - 0s 108us/step - loss: 1.3738 - accuracy: 0.8831 - val_loss: 0.5252 - val_accuracy: 0.8586
Epoch 665/1000
231/231 [==============================] - 0s 102us/step - loss: 1.7816 - accuracy: 0.8528 - val_loss: 0.5260 - val_accuracy: 0.8586
Epoch 666/1000
231/231 [==============================] - 0s 99us/step - loss: 2.1082 - accuracy: 0.8312 - val_loss: 0.5313 - val_accuracy: 0.8586
Epoch 667/1000
231/231 [==============================] - 0s 100us/step - loss: 1.7705 - accuracy: 0.8615 - val_loss: 0.5316 - val_accuracy: 0.8586
Epoch 668/1000
231/231 [==============================] - 0s 95us/step - loss: 1.9766 - accuracy: 0.8355 - val_loss: 0.5280 - va

231/231 [==============================] - 0s 101us/step - loss: 1.6223 - accuracy: 0.8701 - val_loss: 0.5065 - val_accuracy: 0.8788
Epoch 773/1000
231/231 [==============================] - 0s 118us/step - loss: 1.3700 - accuracy: 0.8745 - val_loss: 0.5438 - val_accuracy: 0.8586
Epoch 774/1000
231/231 [==============================] - 0s 113us/step - loss: 2.0302 - accuracy: 0.8355 - val_loss: 0.5469 - val_accuracy: 0.8586
Epoch 775/1000
231/231 [==============================] - 0s 103us/step - loss: 1.5600 - accuracy: 0.8745 - val_loss: 0.5189 - val_accuracy: 0.8687
Epoch 776/1000
231/231 [==============================] - 0s 123us/step - loss: 1.7746 - accuracy: 0.8485 - val_loss: 0.5696 - val_accuracy: 0.8485
Epoch 777/1000
231/231 [==============================] - 0s 99us/step - loss: 1.5671 - accuracy: 0.8701 - val_loss: 0.5406 - val_accuracy: 0.8586
Epoch 778/1000
231/231 [==============================] - 0s 106us/step - loss: 2.2271 - accuracy: 0.8268 - val_loss: 0.5273 - v

231/231 [==============================] - 0s 156us/step - loss: 1.6987 - accuracy: 0.8571 - val_loss: 0.5515 - val_accuracy: 0.8586
Epoch 883/1000
231/231 [==============================] - 0s 105us/step - loss: 1.8158 - accuracy: 0.8615 - val_loss: 0.5805 - val_accuracy: 0.8586
Epoch 884/1000
231/231 [==============================] - 0s 113us/step - loss: 1.4229 - accuracy: 0.8788 - val_loss: 0.5542 - val_accuracy: 0.8586
Epoch 885/1000
231/231 [==============================] - 0s 109us/step - loss: 1.3592 - accuracy: 0.8831 - val_loss: 0.5420 - val_accuracy: 0.8687
Epoch 886/1000
231/231 [==============================] - 0s 105us/step - loss: 1.9501 - accuracy: 0.8485 - val_loss: 0.5387 - val_accuracy: 0.8687
Epoch 887/1000
231/231 [==============================] - 0s 111us/step - loss: 1.4058 - accuracy: 0.8874 - val_loss: 0.5473 - val_accuracy: 0.8687
Epoch 888/1000
231/231 [==============================] - 0s 117us/step - loss: 1.9552 - accuracy: 0.8485 - val_loss: 0.5621 - 

231/231 [==============================] - 0s 115us/step - loss: 1.9573 - accuracy: 0.8398 - val_loss: 0.5832 - val_accuracy: 0.8586
Epoch 993/1000
231/231 [==============================] - 0s 105us/step - loss: 1.4862 - accuracy: 0.8745 - val_loss: 0.5983 - val_accuracy: 0.8586
Epoch 994/1000
231/231 [==============================] - 0s 108us/step - loss: 1.4151 - accuracy: 0.8788 - val_loss: 0.5725 - val_accuracy: 0.8687
Epoch 995/1000
231/231 [==============================] - 0s 97us/step - loss: 1.7479 - accuracy: 0.8615 - val_loss: 0.5551 - val_accuracy: 0.8687
Epoch 996/1000
231/231 [==============================] - 0s 96us/step - loss: 2.0265 - accuracy: 0.8312 - val_loss: 0.5460 - val_accuracy: 0.8687
Epoch 997/1000
231/231 [==============================] - 0s 95us/step - loss: 1.5488 - accuracy: 0.8701 - val_loss: 0.5599 - val_accuracy: 0.8687
Epoch 998/1000
231/231 [==============================] - 0s 104us/step - loss: 1.8117 - accuracy: 0.8615 - val_loss: 0.5728 - val

In [52]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

99/99 [==============================] - 0s 124us/step
over-sampling test accuracy: 86.87%


In [53]:
############ Logistic Regression ############

In [54]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [67]:
###### logistics
log = LogisticRegression()
log.fit(X_train, y_train)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [68]:
y_pred = log.predict(X_test)
print('logistic test accuracy %.2f%%' % (log.score(X_test, y_test)*100))

logistic test accuracy 77.92%


In [55]:
###### logistics on combination data
log_comb = LogisticRegression()
log_comb.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [56]:
y_pred_comb = log_comb.predict(X_test_comb)
print('combination logistic test accuracy %.2f%%' % (log_comb.score(X_test_comb, y_test_comb)*100))

combination logistic test accuracy 98.39%


In [57]:
##### logistics on over-sampling data
log_over = LogisticRegression()
log_over.fit(X_train_over, y_train_over)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [58]:
y_pred_over = log_over.predict(X_test_over)
print('over-sampling logistic test accuracy %.2f%%' % (log_over.score(X_test_over, y_test_over)*100))

over-sampling logistic test accuracy 86.87%


In [59]:
############## Random Forest ##############

In [69]:
###### random forest
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [70]:
from sklearn.metrics import accuracy_score
y_pred = rf.predict(X_test)
print('test accuracy: %.2f%%' % (accuracy_score(y_test, y_pred)*100))

test accuracy: 76.62%


In [60]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100)
rf_comb.fit(X_train_comb,y_train_comb)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [61]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 98.39%


In [62]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [63]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 88.89%


In [64]:
#### Random forest with cross-validation
## Retrieved from https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/

In [71]:
## random forest model with CV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv = RandomForestClassifier(n_estimators=100, random_state=123)

accs = cross_val_score(estimator=rfcv, X=X_train, y=y_train, cv=5)
print(accs)
print(accs.mean())

[0.80555556 0.88888889 0.83333333 0.85714286 0.8       ]
0.836984126984127


In [65]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rfcv_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[1.         1.         0.93103448 0.93103448 1.        ]
0.9724137931034482


In [66]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rfcv_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.89361702 0.84782609 0.82608696 0.73913043 0.89130435]
0.8395929694727104
